In [ ]:
import sys
import Test_Shy.Util
import Env_shy
import numpy as np
from gymnasium.wrappers import TimeLimit
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import bug_lib
import stable_baselines3 as sb3
import os
import gymnasium as gym
import imageio
import bug_lib


In [ ]:
import bug_lib
bug_lib.cover_then_inject_bugs([])

In [ ]:
def generate_gif(model, output_path, num_steps=10000, fps=20):
    images = []
    obs = model.env.reset()
    
    # 渲染初始状态
    img = model.env.render(mode='rgb_array')
    images.append(img)
    
    for i in range(num_steps):
        action, _ = model.predict(obs)
        obs, _, done, _ = model.env.step(action)
        
        img = model.env.render(mode='rgb_array')
        images.append(img)
        
        if done:
            break

    model.env.close()  # 关闭环境
    
    imageio.mimsave(output_path, images, fps=fps)

In [ ]:
# def generate_and_save_matrix(n, m, trial_no, num_arrays):
#     folder_path = f'./saved_array/2by2/{trial_no}/'
#     os.makedirs(folder_path, exist_ok=True)

#     for i in range(num_arrays):
#         A, B = Test_Shy.Util.generate_state_transition_matix(n, m)
#         np.save(f'./saved_array/2by2/{trial_no}/array_A_{i}.npy'.format(trial_no=trial_no, i=i), A)
#         np.save(f'./saved_array/2by2/{trial_no}/array_B_{i}.npy'.format(trial_no=trial_no, i=i), B)

In [1]:
# A2C: [24, 28, 29, 31, 32, 40, 41, 44, 45, 46, 47, 48, 49, 50, 27, 26,6, 10, 52]
# PPO: [54, 28, 55, 56, 57, 58, 44, 45, 46, 47, 48, 49, 50, 27, 26, 6, 10, 52, 59, 60, 61, 62]
# TD3: [53, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]


# TD3: [53, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]
bug_no = 70

In [4]:
import bug_lib


# A2Clib    [24, 28, 29, 31, 32, 40, 41, 44, 45, 46, 47, 48, 49, 50, 27, 26, 6, 10, 52]
# A2C clean [7, 12, 15, 18, 20, 33, 35, 37, 39, 42, 51, 54, 56, 58, 60, 62, 64, 66, 68]


# PPO:           [54, 28, 55, 56, 57, 58, 44, 45, 46, 47, 48, 49, 50, 27, 26, 6, 10, 52, 59, 60, 61, 62]
# PPO bugfree no:[37, 12, 81, 93, 76, 42, 19, 33, 67, 85, 71, 24, 39, 8, 16, 2, 90, 53, 69, 74, 31, 22]

if bug_no != 'free':
    bug_lib.cover_then_inject_bugs([bug_no])

from typing import Any, ClassVar, Dict, List, Optional, Tuple, Type, TypeVar, Union

import numpy as np
import torch as th
from gymnasium import spaces
from torch.nn import functional as F

from stable_baselines3.common.buffers import ReplayBuffer
from stable_baselines3.common.noise import ActionNoise
from stable_baselines3.common.off_policy_algorithm import OffPolicyAlgorithm
from stable_baselines3.common.policies import BasePolicy, ContinuousCritic
from stable_baselines3.common.type_aliases import GymEnv, MaybeCallback, Schedule
from stable_baselines3.common.utils import get_parameters_by_name, polyak_update
from stable_baselines3.td3.policies import Actor, CnnPolicy, MlpPolicy, MultiInputPolicy, TD3Policy

SelfTD3 = TypeVar("SelfTD3", bound="TD3")


class TD3(OffPolicyAlgorithm):
    """
    Twin Delayed DDPG (TD3)
    Addressing Function Approximation Error in Actor-Critic Methods.

    Original implementation: https://github.com/sfujim/TD3
    Paper: https://arxiv.org/abs/1802.094

In [2]:
import sys
import Test_Shy.Util
import Env_shy
import numpy as np
from gymnasium.wrappers import TimeLimit
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import bug_lib
import stable_baselines3 as sb3
import os
import gymnasium as gym
import imageio


def generate_gif(model, output_path, num_steps=10000, fps=20):
    images = []
    obs = model.env.reset()
    
    # 渲染初始状态
    img = model.env.render(mode='rgb_array')
    images.append(img)
    
    for i in range(num_steps):
        action, _ = model.predict(obs)
        obs, _, done, _ = model.env.step(action)
        
        img = model.env.render(mode='rgb_array')
        images.append(img)
        
        if done:
            break

    model.env.close()  # 关闭环境
    
    imageio.mimsave(output_path, images, fps=fps)

def run_one_trial(env, algorithm, trial_no, bug_no):
    learning_steps = (trial_no + 1) * 10000
    # learning_steps = 500

    #bug free log multiple steps
    file_path_log = './TensorBoard_Logs/human_oracle/env_{env}/alg_{alg}/version_no_{bug_no}/learning_steps_{learning_steps}'.format(env=env, trial_no=trial_no, bug_no=bug_no, alg=algorithm, learning_steps=learning_steps)

    # for i in range(1):
    if env == 'MountainCarContinuous':
        env = gym.make('MountainCarContinuous-v0', render_mode="rgb_array")
    elif env =='Pendulum':
        env = gym.make('Pendulum-v1', g=9.81, render_mode="rgb_array")

    if algorithm == 'ppo':
        model = sb3.PPO("MlpPolicy", env, verbose=1, tensorboard_log=file_path_log)
        model.learn(total_timesteps=learning_steps)
    elif algorithm == 'a2c':
        model = sb3.A2C("MlpPolicy", env, verbose=1, tensorboard_log=file_path_log)
        model.learn(total_timesteps=learning_steps)
    elif algorithm == 'td3':
        model = sb3.TD3("MlpPolicy", env, verbose=1, tensorboard_log=file_path_log)
        model.learn(total_timesteps=learning_steps)

    model.save(file_path_log + 'model')
    generate_gif(model, file_path_log + 'after_training.gif')

num_trails = 3
algorithm = 'td3'
# env = 'Pendulum'
# env = 'MountainCarContinuous'
for env in ['Pendulum', 'MountainCarContinuous']:
    for i in range(num_trails):
        run_one_trial(env=env, algorithm=algorithm, trial_no=i, bug_no=bug_no)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./TensorBoard_Logs/human_oracle/env_Pendulum/alg_td3/version_no_70/learning_steps_10000\TD3_1
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.51e+03 |
| time/              |           |
|    episodes        | 4         |
|    fps             | 71        |
|    time_elapsed    | 11        |
|    total_timesteps | 800       |
| train/             |           |
|    actor_loss      | 17.7      |
|    critic_loss     | 0.0245    |
|    learning_rate   | 0.001     |
|    n_updates       | 699       |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.46e+03 |
| time/              |           |
|    episodes        | 8         |
|    fps             | 67        |
|    time_elapsed    | 23      

KeyboardInterrupt: 

In [ ]:
%tensorboard --logdir TensorBoard_Logs/human_oracle/

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --version